
<center>
    <h1> INF285/ILI285 Computación Científica </h1>
    <h1> Tarea N°1, v1.00 </h1>
    
</center>

<p>

</p>

## Instrucciones

* La tarea es individual.
* Las consultas sobre las tareas se deben realizar por medio de la plataforma Aula.
* La tarea debe ser realizada en `Jupyter Notebook` (`Python3`).
* Se evaluará la correcta utilización de librerias `NumPy`, `SciPy`, entre otras, así como la correcta implementación de algoritmos de forma vectorizada.
*  **El archivo de entrega debe denominarse ROL-tarea-numero.ipynb**. _De no respetarse este formato existirá un descuento de **50 puntos**_
* La fecha de entrega es el jueves 30 de Abril a las **18:00 hrs**.  Se aceptarán entregas hasta las 19:00 hrs sin descuento en caso de existir algun problema, posteriormente existirá un descuento lineal hasta las 20:00 hrs del mismo día.
* Las tareas que sean entregadas antes del jueves a mediodía recibirán una bonificación de 10 puntos
* Debe citar cualquier código ajeno utilizado (incluso si proviene de los Jupyter Notebooks del curso).




## Introducción 

En esta primera tarea de INF/ILI-285, versión 2020-1, estudiaremos la importancia de los primeros temas estudiados en el curso, los cuales son: Representación de Punto Flotante, Pérdida de Significancia, Errores de Cancelación y Búsqueda de Ceros. El desarrollo de cada uno de esos temas se presenta en una serie de preguntas, donde ustedes deben ir decidiendo, pregunta a pregunta, como cada uno de los temas se aplica. En general, los temas no se analizan por separado, sino de manera acoplada. Es muy importante que cada uno de los problemas sea analizado teóricamente primero, para luego poner su conocimiento en acción. Cada problema puede ser desarrollado de diversas formas, sin embargo, es muy importante determinar al final si el camino elegido resuelve la pregunta presentada.

## Problemas

### 1. Simulador (50 ptos) 

Dada la familia de polinomios de grado 3 con la forma:


\begin{equation}
f(x)=x^3 - A\,x^2 + A\,x - 1
\end{equation}


Se pide implementar un algoritmo que reciba como parámetros el valor de $A$ (con $|A|>>1$) y la cantidad de bits que tiene en la mantisa un computador ficticio, el manejo del exponente debe considerarse de las misma forma que lo maneja _double precision_.
Esta implementación debe calcular las raíces de $f$ sin perdida de significancia con la cantidad de bits disponibles para la mantisa.
Para obtener las raíces de $f$ usted debe encontrar de forma algebraica sus raíces y luego proponer un algoritmo basado en las _fórmulas_ obtenidas.

Considere que en ese computador ficticio cuenta con las operaciones matemáticas necesarias para obtener las raíces.  Considere el siguiente ejemplo:

```python
# Alg. Base
a = 9819824.624837
b = 148736.523476
c = a+b

# Alg. con Representación de Punto Flotante de 'bits_mant' bits en la mantisa.
am = f_new_rep(9819824.624837,bits_mant) # Aproximar el input en la nueva representación.
bm = f_new_rep(148736.523476,bits_mant) # Aproximar el input en la nueva representación.
cm = f_new_rep(m,am+bm) # Aproximar el output de la suma y cada operación en la nueva representación.
```


A continuación se desarrolla la función *f_new_rep* la cual hace uso de distintas funciones de apoyo para transformar el número a binario con la mantisa entregada y luego volver a pasarlo a decimal con esa mantisa:

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import math
import scipy

In [140]:
"""
input
x         : (double) valor a evaluar
bits_mant : (int) cantidad de bits de la mantisa
output
eval      : (double) resultado obtenido
"""

#convierte la parte entera del numero recibido a binario
def convertirEntera(numero):
    numero=int(numero)
    flag=True
    binario=""
    while flag:
        R=numero%2
        numero=int(numero/2)
        if(R==1):
            binario=binario+"1"
        else:
            binario=binario+"0"
        if numero==0:
            flag=False
    #print(binario[::-1])
    return binario[::-1]

#convierte la parte decimal del numero recibido a binario
def convertirDecimal(numero):
    mantisa=0
    binario=""
    numero=float("0."+numero)
    repetidos=[]
    flag=True
    while flag:
        numero=numero*2
        if numero>=1:
            numero=numero-1
            binario="1"+binario
            if numero in repetidos:
                flag=False
            repetidos.append(numero)
        else:
            if numero in repetidos:
                flag=False
            repetidos.append(numero)
            binario="0"+binario

    #print(binario[::-1])
    return binario[::-1]
#recibe un binario y le suma 1 
def suma(bin):
    largoBin=len(bin)
    resultado=""
    pos=0
    if bin[-1]=="0":
        #print(bin)
        resultado=bin[0:largoBin-1]+"1"
        #print(resultado)
        return resultado
    else:#como el ultimo es 1 hay que sumarle 1 por lo que queda 0 y genera carry
        bin=bin[::-1]
        for bit in bin:
            if bit == "1":
                pos+=1
                resultado=resultado+"0"
            else:
                resultado=resultado+"1"
                resultado=resultado+bin[pos+1:]
                return resultado[::-1]
#recibe un numero para pasarlo a decimal, si es 0 o 1 sirve para diferenciar si es pasar la parte entera o decimal
def pasarDecimal(numero,estado):
    if estado==0:
        resultado=0
        i=1
        for bit in numero:
            resultado=resultado+(2**(-1*i))*int(bit)
            i+=1
        return resultado
    else:  
        resultado=0
        i=0
        for bit in numero[::-1]:
            resultado=resultado+(2**(i))*int(bit)
            i+=1
        return resultado
#funcion que recibe un float y devuelve un float con la mantisa entregada
def f_new_rep(x,bits_mant):
    if x==0.0:
        return 0.0
  # Algoritmo de representación de punto flotante modificada.
    signo=0
    exponente=0
    if x<0:
        signo=1
    x=format(x,'.52f')
    a=str(x)
    l=a.split(".")
    binarioEntero=str(abs(int(convertirEntera(l[0]))))
                                            #esto se hace porque cuando es periodico el numero puede quedar muy corto
    binarioDecimal=convertirDecimal(l[1])+"000000000000000000000000000000000000000000000000000000000000000000000000000"
    nuevaRepresentacion=""
    
    if binarioEntero == "0":
        exponente=(binarioDecimal.find("1")+1)*-1
        nuevaRepresentacion="1."+binarioDecimal[binarioDecimal.find("1")+1:]

        if nuevaRepresentacion[(bits_mant+1)+1]=="0": #si el ultimo bit +1 que considero es 0
            nuevaRepresentacion=nuevaRepresentacion[0:bits_mant+2]
            #print("final0: "+nuevaRepresentacion)
        else: #si el ultimo bit q considero es 1
 
            if nuevaRepresentacion[bits_mant+2:].find("1") != -1:    #si despues del ultimo bit hay algun 1 
                                                                     #entonces redondeamos hacia arriba

                nuevaRepresentacion=nuevaRepresentacion[0:bits_mant+2]
     
                nuevaRepresentacion="1."+suma(nuevaRepresentacion[2:])
    
            else: #se suma 1 al bit 52 solo si este bit es 1

                if nuevaRepresentacion[bits_mant+1]=="1":
                    nuevaRepresentacion="1."+suma(nuevaRepresentacion[2:])
                    
    else: 
        exponente=len(binarioEntero)-1
        nuevaRepresentacion="1."+binarioEntero[1:]+binarioDecimal
   
        
        #casos mantisa
        if nuevaRepresentacion[(bits_mant+1)+1]=="0": #si el ultimo bit +1 que considero es 0
            nuevaRepresentacion=nuevaRepresentacion[0:bits_mant+2]
           
        else: #si el ultimo bit q considero es 1
 
            if nuevaRepresentacion[bits_mant+2:].find("1") != -1:   #si despues del ultimo bit hay algun 1 
                                                                    #entonces redondeamos hacia arriba
               
                nuevaRepresentacion=nuevaRepresentacion[0:bits_mant+2]
             
                nuevaRepresentacion="1."+suma(nuevaRepresentacion[2:])
                
                
            else: #se suma 1 al bit 52 solo si este bit es 1
                if nuevaRepresentacion[bits_mant+1]=="1":
                    nuevaRepresentacion="1."+suma(nuevaRepresentacion[2:])
    
    nuevoBin=""
    if exponente>0:
        
        j=exponente-(len(nuevaRepresentacion)-2) #parar correr el exponente
        #print(j)
        if j>0:
            i=0
            while(i<j):
                nuevaRepresentacion=nuevaRepresentacion+"0"
                i=i+1

        nuevoBin="1"+nuevaRepresentacion[2:exponente+2]+"."+nuevaRepresentacion[exponente+2:]
        if nuevoBin[-1]==".":
            nuevoBin=nuevoBin+"0"
        #print(nuevoBin)
        l=nuevoBin.split(".")
        parteDecimal=pasarDecimal(l[1],0)
        parteEntera=pasarDecimal(l[0],1)
        
    else:
        i=1
        nuevoBin="0."
        while(i<exponente*-1):
            nuevoBin=nuevoBin+"0"
            i+=1
        nuevoBin=nuevoBin+"1"+nuevaRepresentacion[2:]
        l=nuevoBin.split(".")
  
        parteDecimal=pasarDecimal(l[1],0) # 0 es para avisarle a la func q es la parte decimal
        parteEntera=pasarDecimal(l[0],1)
    result=parteEntera+parteDecimal

    if signo==0:
        return float(result)
    else:
        return float(result*-1)
#Hubo demasiado esfuerzo en esta pregunta porfavor piedad u.u


Podemos observar que x=1 es una raíz del polinomio, por lo que podemos utilizar una división de polinomios para encontrar las demás raíces:


<div align="center"> $x^3-Ax^2+Ax-1  : (x-1)  =    x^2+(1-A)x+1$ <div/>

Encontramos que el polinomio $x^2+(1-A)x+1$ nos sirve para encontrar 2 de las raices restantes del polinomio original ya que tenemos la raiz x=1.

Para resolver la función cuadrática se utilizará la fórmula cuadrática tradicional:

<div align="center">$\frac{-b+-\sqrt{b^2-4ac}}{2a}$<div/>


Sin embargo, para evitar pérdida de significancia hay que evitar la operación de resta en la fórmula. Para evitar restas es necesario ponerse en los distintos casos de b (1-A), ya que como podemos observar a=1, b=1-A y c=1, lo cual nos indica que para que no existan restas depende del signo que adopte A. De esta forma, en algunos casos se podrá aplicar la fórmula cuadrática tradicional directamente y en otros se tendrá que trabajar algebraicamente de la siguiente forma para evitar las restas.
    

Expondremos un caso que es cuando la fórmula adopta la suma (el caso es homólogo en la resta):
<div align="center">$\frac{-b+\sqrt{b^2-4ac}}{2a}  $<div/>
    
Racionalizando:   
<div align="center">$\frac{-b+\sqrt{b^2-4ac}}{2a} \frac{b+\sqrt{b^2-4ac}}{b+\sqrt{b^2-4ac}} $<div/>

Finalmente:
<div align="center">$\frac{-2c}{b+\sqrt{b^2-4ac}}  $<div/>

In [163]:
#recibe el número A y la mantisa y regresa las raices de la función para ese A
def f_find_roots(A,bits_mant):
    # Algoritmo para encontrar raíces de f(x).
    #notar que a=1, b=1-A y c=1
    #por lo que -b=A-1
    AmenosUno=f_new_rep(A-1,bits_mant)
    AmenosUnoCuadrao=f_new_rep(AmenosUno*AmenosUno,bits_mant)
    AmenosUnoCuadraoMenosCuatro= f_new_rep(AmenosUnoCuadrao-4,bits_mant)
    raiz=f_new_rep((AmenosUnoCuadraoMenosCuatro)**0.5,bits_mant)
    ######################## bien: separar casos para evitar restas
    if AmenosUno<0: #si a menos uno es negativo
        AmenosUno=AmenosUno*-1
        numeradorx1=f_new_rep( AmenosUno+raiz ,bits_mant)
        x1=f_new_rep( (numeradorx1) / (2) ,bits_mant)*-1
        denominadorx2=f_new_rep((AmenosUno+raiz) ,bits_mant)
        x2=f_new_rep((-2)/(denominadorx2),bits_mant)
        x_roots=[]
        x_roots.append(1)
        x_roots.append(x1)
        x_roots.append(x2)
        return x_roots
    else:
        numeradorx2=f_new_rep(AmenosUno+raiz,bits_mant)
        x2=f_new_rep(numeradorx2/2,bits_mant)
        denominadorx1=f_new_rep(AmenosUno+raiz,bits_mant)
        x1=f_new_rep(2/denominadorx1,bits_mant)
        x_roots=[]
        x_roots.append(1)
        x_roots.append(x1)
        x_roots.append(x2)
        return x_roots
    

In [164]:
print("FIND ROOTS: ",f_find_roots(50000.125, 10) )

FIND ROOTS:  [1, 2.0012259483337402e-05, 49984.0]


### Polinomios de Legendre (50 puntos)
Dada la función compuesta $f$
\begin{equation}
f_{n,m}(x) = L_n(C_m(x)),
\end{equation}
donde $L_n$ es conocido como el polinomio de Legendre de grado $n$ definido de la siguiente forma:
\begin{equation}
L_{n}(x)=\frac{1}{2^{n}} \sum_{k=0}^{n}\left(\begin{array}{l}
n \\
k
\end{array}\right)^{2}(x-1)^{n-k}(x+1)^{k},
\end{equation}
y $C_m$ es el polinomio de Chebyshev
\begin{equation}
C_m(x) = \cos(m \cdot \arccos(x)).
\end{equation}

Utilizando el método de Bisección y Punto fijo se pide obtener la raíz de $f$ más cercana a $0.5$ dado un valor de $m$ y $n$



*Hint: Las raíces de Legendre son conocidas*



In [3]:
"""
input
n: (int) grado del polinomio de Legendre
m: (int) grado del polinomio de Chebyshev
tol: (double) tolerancia para la detención del algoritmo
output
root: (double) raiz obtenida
"""
def f_Biseccion(n, m, tol):
    ##
    return root

def f_FPI(n, m, tol):
    return root

###  Determinantes (20 puntos)

Dada una matriz de dimensiones $ n \times n$ de la forma:
\begin{equation}
A
=
\begin{pmatrix}
a_{1,1} & a_{1,2}  & \dots & a_{1,n} \\
a_{2,1} & a_{2,2}  & \dots & a_{2,n} \\
\vdots &  \vdots   & \ddots & \vdots \\
a_{n,1} & a_{n,2}  & \dots & a_{n,n}
\end{pmatrix}
=
\begin{pmatrix}
\mathbf{r}_1 \\
\mathbf{r}_2 \\
\vdots \\
\mathbf{r}_n \\
\end{pmatrix},
\end{equation}
donde $\mathbf{r}_k$ representa la $k$-ésima fila de la matriz $A$.
Considere la siguiente matriz $C_{i,j}(\alpha)$,
\begin{equation}
C_{i,j}(\alpha)
=
\begin{pmatrix}
\mathbf{r}_1 \\
\vdots \\
\mathbf{r}_i\,(1-\alpha)+\mathbf{r}_j\,\alpha \\
\vdots \\
\mathbf{r}_j\,(1-\alpha)+\mathbf{r}_i\,\alpha \\
\vdots \\
\mathbf{r}_n \\
\end{pmatrix},
\end{equation}
de lo cual sabemos que $C_{i,j}(0)=A$ y que $C_{i,j}(1)$ es la matriz $A$ donde se intercambiaron las filas $i$ y $j$, es decir:
\begin{equation}
C_{i,j}(1)
=
\begin{pmatrix}
\mathbf{r}_1 \\
\vdots \\
\mathbf{r}_j \\
\vdots \\
\mathbf{r}_i \\
\vdots \\
\mathbf{r}_n \\
\end{pmatrix}.
\end{equation}
De las relaciones anteriores podemos concluir que el determinante de la matriz $A$, denominado $D=\det(A)$, es igual al determinante de $C_{i,j}(0)$, es decir $\det(C_{i,j}(0))=\det(A)=D$.
Por el otro lado, el determinante de $C_{i,j}(1)$ es $-D$, dado que es el intercambio de las filas $i$ y $j$ de la matriz $A$.
Por lo cual podemos concluir que $-D\leq \det(C_{i,j}(\alpha))\leq D$.

Utilizando el método de Bisección debe encontrar el valor de $\alpha$ con $p$ decimales de precisión que permitan que, dado los índices de las filas $i$, $j$, y $i\neq j$, el determinante de la matriz sea igual a $d$, donde $d\in[-D,D]$. 

Para esto se debe implementar una función que reciba la matriz $A$, las filas $i$ y $j$, y $p$; y retorne $\widehat{\alpha}$ tal que $\det(C_{i,j}(\widehat{\alpha}))=d$.


La idea es aplicar bisección en el intervalo 0 y 1 e ir variando los valores de acuerdo al valor de los determinantes que cambiarán segun el alfa evaluado:

In [16]:
"""
input
A: (array n x n) matriz
i: (int) índice de la fila "i".
j: (int) índice de la fila "j".
p: (int) cantidad de decimales de precision 
d: (double) valor requerido del determinante de $C_{i,j}(\alpha)$, i.e. $\det(C_{i,j}(\widehat{\alpha}))=d$.
output
alpha_hat: (double) alpha_hat tal que det(C_{i,j}(alpha_hat))=d.
"""
def find_alpha_hat(A,i,j,p,d):
    # Algoritmo a desarrollar, note que usted debe definir el intervalo a utilizar en el método de la bisección.
    a=0
    b=1
    cont=0
    n=np.log2(1/(0.5*(10**-p)))-1
    detA=d-np.linalg.det(A)
    detB=-detA
    f_a=d-detA
    f_b=d+detB
    #caso borde
    if abs(d)>=abs(detA):
        if np.sign(d*detA)>0:
             return 0 
        else:
            return 1
    while(cont<n):
        #print(cont)
        #print(n)
        alfa=(a+b)/2
        C = np.copy(A)
        #generamos la matriz C donde hay que reemplazar las filas r_i y r_j
        fila_i = np.copy(C[i])
        fila_j = np.copy(C[j])
        C[i]= fila_j*(alfa)+fila_i*(1-alfa) #reemplazamos
        C[j]= fila_i*(alfa)+fila_j*(1-alfa)
        f_alfa = d - np.linalg.det(C)
        if f_alfa < 0.5*(10**-p):
            alpha_hat=alfa
            return round(alpha_hat)
        if f_alfa*f_a<0:
            b=alfa
            f_b=f_alfa
        else:
            a=alfa
            f_a=f_alfa
        cont+=1
    alpha_hat=alfa        
    return round(alpha_hat)


In [19]:
A = np.array([[25,100,100],[20,50,20],[40,40,40]])
B = np.array([[2,1,3],[2,5,1],[4,1,10]])
C = np.array([[25,100,10.0,30],[20,50,20,-14],[40,40,40,100],[1,-1,1,-1]])
D = np.array([[25,100,10.0,30,-100],[20,50,20,-14,-20],[40,40,40,100,100],[1,-1,1,-1,-2],[1,2,3,4,5]])

d_values=[9,46599,-8319,89000] # B, C, D, A respectivamente
#m=alguna de las cuatro matrices
p=10
i=0
j=A.shape[1]-1
#Se intercambiaban la primera fila con la ultima
find_alpha_hat(D,i,j,p,d_values[2])

0

# Referencias

Apuntes y jupyters del curso (nada textual)